In [ ]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("yasserh/breast-cancer-dataset")

print("Path to dataset files:", path)

In [ ]:
df=pd.read_csv("breast-cancer.csv")
df.head()



In [ ]:
df.drop(columns="id",inplace=True)

In [ ]:
df.head()

In [ ]:
df.shape

train test split

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(df.iloc[:,1:],df.iloc[:,0],test_size=0.2)

Scaling

In [ ]:
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.fit_transform(X_test)

In [ ]:
encoder=LabelEncoder()
y_train=encoder.fit_transform(y_train)
y_test=encoder.fit_transform(y_test)

In [ ]:
y_train


Numpy to Pytorch

In [ ]:
X_train_tensor=torch.from_numpy(X_train)
X_test_tensor=torch.from_numpy(X_test)
y_train_tensor=torch.from_numpy(y_train)
y_test_tensor=torch.from_numpy(y_test)

In [ ]:
X_train_tensor


In [ ]:
y_train_tensor.shape

Defining The Model


In [ ]:
class MySimpleNN():
    def __init__(self,X):
        self.weights=torch.rand(X.shape[1],1,dtype=torch.float64,requires_grad=True)
        self.bias=torch.zeros(1,dtype=torch.float64,requires_grad=True)
    def forward(self,X):
        z=torch.matmul(X,self.weights)+self.bias
        y_pred=torch.sigmoid(z)
        return y_pred
    def loss_function(self,y_pred,y):
        epsilon=1e-7
        y_pred=torch.clamp(y_pred,epsilon,1-epsilon)
        loss=-(y_train_tensor*torch.log(y_pred)+(1-y_train_tensor)*torch.log(1-y_pred)).mean()
        return loss


In [ ]:
learning_rate=0.1
epochs=25


In [ ]:
#create model
model=MySimpleNN(X_test_tensor)
#forward pass
for epoch in range(epochs):
    y_pred=model.forward(X_train_tensor)
    #loss calculate
    loss=model.loss_function(y_pred,y_train_tensor  )
    #backward pass
    loss.backward() 
    #parameter update
    with torch.no_grad():
        model.weights-=learning_rate*model.weights.grad 
        model.bias-=learning_rate*model.bias.grad

    model.weights.grad.zero_()
    model.bias.grad.zero_()

In [ ]:
model.weights

In [ ]:
model.bias

In [ ]:
#model evaluation
with torch.no_grad():
    y_pred=model.forward(X_test_tensor)
    y_pred=(y_pred>0.5).float()
    accuracy=(y_pred==y_test_tensor).float().mean()
    print(f'Accuracy:{accuracy.item()}')